In [1]:
from pathlib import Path

import numpy as np
#import pytest

from abacusnbody.analysis.power_spectrum import calc_power

_curdir = Path('/Users/jyotsnaravi/LAEs in MTNG/data/z = 2/galaxy attributes/SubhaloPos_fp_129.npy').parent
DATA_POWER = _curdir / 'data_power'


power_test_data = {
    "Lbox": 500.0,
    "pos": np.load('/Users/jyotsnaravi/LAEs in MTNG/data/z = 2/galaxy attributes/SubhaloPos_fp_129.npy'),
}

res = calc_power(
    power_test_data['pos'],
    power_test_data['Lbox'],
    interlaced=True,
    compensated=True,
    paste='TSC'
)

print(res['power'])


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


   power    
------------
4.536586e-06
   5509.2935
    5470.544
    4952.895
    4792.464
   3920.0586
   3470.2393
   2903.2603
   2807.0498
   2478.0679
         ...
   45.843563
   44.543266
   44.163826
   43.167404
    43.26461
    42.37794
      41.691
   41.243073
   40.697876
   40.482494
Length = 128 rows


In [2]:
subhalo_mass = np.load('/Users/jyotsnaravi/MTNG IA/data_214/SubhaloMassType_fp_214.npy')

In [3]:
stellar_mass = subhalo_mass[:, 4]

In [4]:
volume_box = 500 ** 3
num_density = 10 ** (-3)
num_galaxies = int(volume_box * num_density)

In [5]:
mass_indicies = np.argsort(stellar_mass)[-num_galaxies:]

In [58]:
len(mass_indicies) / volume_box

0.001

In [7]:
pos = np.load('/Users/jyotsnaravi/MTNG IA/data_214/SubhaloPos_fp_214.npy')
pos_mass_selected = pos[mass_indicies]

In [8]:
pos_mass_selected.shape

(125000, 3)

In [9]:
from abacusnbody.analysis.tsc import tsc_parallel
N_gal = tsc_parallel(pos_mass_selected, np.zeros((512, 512, 512), dtype=np.float32), 500)

In [10]:
inertia_tensor = np.load('/Users/jyotsnaravi/MTNG IA/data_214/SubhaloIntertiaTensorStars_fp_214.npy')

In [11]:
inertia_xx = inertia_tensor[:, 0][mass_indicies]
inertia_xy = inertia_tensor[:, 1][mass_indicies]
inertia_xz = inertia_tensor[:, 2][mass_indicies]
inertia_yy = inertia_tensor[:, 3][mass_indicies]
inertia_yz = inertia_tensor[:, 4][mass_indicies]
inertia_zz = inertia_tensor[:, 5][mass_indicies]

In [12]:
I_xx = tsc_parallel(pos_mass_selected, np.zeros((512, 512, 512), dtype=np.float32), 500, weights=inertia_xx)

In [13]:
I_xy = tsc_parallel(pos_mass_selected, np.zeros((512, 512, 512), dtype=np.float32), 500, weights=inertia_xy)  
I_xz = tsc_parallel(pos_mass_selected, np.zeros((512, 512, 512), dtype=np.float32), 500, weights=inertia_xz)
I_yy = tsc_parallel(pos_mass_selected, np.zeros((512, 512, 512), dtype=np.float32), 500, weights=inertia_yy)
I_yz = tsc_parallel(pos_mass_selected, np.zeros((512, 512, 512), dtype=np.float32), 500, weights=inertia_yz)
I_zz = tsc_parallel(pos_mass_selected, np.zeros((512, 512, 512), dtype=np.float32), 500, weights=inertia_zz)

In [72]:
np.max(I_xx[I_xx !=0])

np.float32(129.66322)

In [14]:
N_gal_I_ij = np.zeros((6,) + (512, 512, 512), dtype=np.float32)
N_gal_I_ij[0, :, :, :] = I_xx
N_gal_I_ij[1, :, :, :] = I_xy
N_gal_I_ij[2, :, :, :] = I_xz
N_gal_I_ij[3, :, :, :] = I_yy
N_gal_I_ij[4, :, :, :] = I_yz
N_gal_I_ij[5, :, :, :] = I_zz

In [15]:
#I_ij = np.divide(N_gal_I_ij, N_gal, out=np.zeros_like(N_gal_I_ij), where=(N_gal != 0))

In [16]:
def Y_00():
    return np.sqrt(1/3) * np.identity(3)

def Y_20(k_hat):
    return np.sqrt(3/2) * (np.outer(k_hat, k_hat) - np.identity(3)/3)

def Y_21(e_basis, k_hat):
    return np.sqrt(1/2) *(np.outer(k_hat, e_basis) + np.outer(k_hat, e_basis))

def Y_22(e_basis):
    return np.outer(e_basis, e_basis)

In [17]:
#I_ij_without_trace = I_ij - (1/3) * np.trace

In [18]:
#1. division by mean galaxies in voxel (check)
# convert to k-space (meshgrid thing) (check)
#2. shape field normalization (2.41) (check)
#3. taking out the trace to get trace-free part (check)
#transfer to helicity space to get the power-spectrum 
#eq. 2,5 to get the helicity basis (where n is the LOS (z-direction) except for the I_ijs that are (0, 0, k_z), where we'll set our n to be (1, 00))
#convert from ij to lm
#then find Pm(ll') for everything

In [19]:
import sys
print(sys.executable)

/Users/jyotsnaravi/miniconda3/bin/python


In [20]:
#pick random seed, pick arbitrary x, y, z, normalize, and use this for n_hat
#next wednesday at 10 am meeting

In [21]:
k_space_bins = np.linspace(0, 1, 41)
L_box = 500
num_bins = 512

kx = np.fft.fftfreq(num_bins, d=L_box / num_bins) * 2*np.pi
ky = np.fft.fftfreq(num_bins, d=L_box / num_bins) * 2*np.pi
kz = np.fft.rfftfreq(num_bins, d=L_box / num_bins) * 2*np.pi
kx, ky, kz = np.meshgrid(kx, ky, kz, indexing='ij')
k_sq = kx**2 + ky**2 + kz**2
k = np.sqrt(k_sq)

k_flat = k.flatten()


"""I_xx_power_3d_bool = (I_xx_power_3d[:, :, 0] == 0) & (I_xx_power_3d[:, :, 1] == 0)
indices_xx = np.argwhere(I_xx_power_3d_bool)


I_xy_power_3d_bool = (I_xy_power_3d[:, :, 0] == 0) & (I_xy_power_3d[:, :, 1] == 0)
indices_xy = np.argwhere(I_xy_power_3d_bool)


I_xz_power_3d_bool = (I_xz_power_3d[:, :, 0] == 0) & (I_xz_power_3d[:, :, 1] == 0)
indices_xz = np.argwhere(I_xz_power_3d_bool)


I_yy_power_3d_bool = (I_yy_power_3d[:, :, 0] == 0) & (I_yy_power_3d[:, :, 1] == 0)
indices_yy = np.argwhere(I_yy_power_3d_bool)


I_yz_power_3d_bool = (I_yz_power_3d[:, :, 0] == 0) & (I_yz_power_3d[:, :, 1] == 0)
indices_yz = np.argwhere(I_yz_power_3d_bool)


I_zz_power_3d_bool = (I_zz_power_3d[:, :, 0] == 0) & (I_zz_power_3d[:, :, 1] == 0)
indices_zz = np.argwhere(I_zz_power_3d_bool)

            
I_xx_flattened = I_xx_power_3d.flatten()
I_xy_flattened = I_xy_power_3d.flatten()
I_xz_flattened = I_xz_power_3d.flatten()
I_yy_flattened = I_yy_power_3d.flatten()
I_yz_flattened = I_yz_power_3d.flatten()
I_zz_flattened = I_zz_power_3d.flatten()"""

'I_xx_power_3d_bool = (I_xx_power_3d[:, :, 0] == 0) & (I_xx_power_3d[:, :, 1] == 0)\nindices_xx = np.argwhere(I_xx_power_3d_bool)\n\n\nI_xy_power_3d_bool = (I_xy_power_3d[:, :, 0] == 0) & (I_xy_power_3d[:, :, 1] == 0)\nindices_xy = np.argwhere(I_xy_power_3d_bool)\n\n\nI_xz_power_3d_bool = (I_xz_power_3d[:, :, 0] == 0) & (I_xz_power_3d[:, :, 1] == 0)\nindices_xz = np.argwhere(I_xz_power_3d_bool)\n\n\nI_yy_power_3d_bool = (I_yy_power_3d[:, :, 0] == 0) & (I_yy_power_3d[:, :, 1] == 0)\nindices_yy = np.argwhere(I_yy_power_3d_bool)\n\n\nI_yz_power_3d_bool = (I_yz_power_3d[:, :, 0] == 0) & (I_yz_power_3d[:, :, 1] == 0)\nindices_yz = np.argwhere(I_yz_power_3d_bool)\n\n\nI_zz_power_3d_bool = (I_zz_power_3d[:, :, 0] == 0) & (I_zz_power_3d[:, :, 1] == 0)\nindices_zz = np.argwhere(I_zz_power_3d_bool)\n\n            \nI_xx_flattened = I_xx_power_3d.flatten()\nI_xy_flattened = I_xy_power_3d.flatten()\nI_xz_flattened = I_xz_power_3d.flatten()\nI_yy_flattened = I_yy_power_3d.flatten()\nI_yz_flattened 

In [22]:
#I_xx_power_3d[:, 0].shape

In [23]:
L_box = 500
num_bins = 512

#I_ij_normalized = I_ij / (len(mass_indicies) / (num_bins ** 3))
I_xx_normalized = I_xx / (len(mass_indicies) / (num_bins ** 3)) #normalization is per voxel
I_xy_normalized = I_xy / (len(mass_indicies) / (num_bins ** 3))
I_xz_normalized = I_xz / (len(mass_indicies) / (num_bins ** 3))
I_yy_normalized = I_yy / (len(mass_indicies) / (num_bins ** 3))
I_yz_normalized = I_yz / (len(mass_indicies) / (num_bins ** 3))
I_zz_normalized = I_zz / (len(mass_indicies) / (num_bins ** 3))

0.0009313225746154785

In [63]:
num_galaxies

125000

In [62]:
mass_indicies

array([5750495, 5488765, 5666401, ...,   42596,   13354,   27463])

In [31]:
np.max(I_xx_normalized[I_xx_normalized != 0])

np.float32(139224.83)

In [50]:
np.min(I_xx_normalized[I_xx_normalized != 0]) #no negatives--makes sense

np.float32(6.7622674e-16)

In [49]:
I_xx_normalized

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [26]:
mean_xx = np.mean(I_xx)
mean_yy = np.mean(I_yy)
mean_zz = np.mean(I_zz)


I_ij_trace = mean_xx + mean_yy + mean_zz
I_xx_shape_normalization = (I_xx - mean_xx) / I_ij_trace

mean_xy = np.mean(I_xy)
I_xy_shape_normalization = (I_xy - mean_xy) / I_ij_trace


mean_xz = np.mean(I_xz)
I_xz_shape_normalization = (I_xz - mean_xz) / I_ij_trace


I_yy_shape_normalization = (I_yy - mean_yy) / I_ij_trace


mean_yz = np.mean(I_yz)
I_yz_shape_normalization = (I_yz - mean_yz) / I_ij_trace


I_zz_shape_normalization = (I_zz - mean_zz) / I_ij_trace

In [77]:
np.min(I_xx_shape_normalization) #very small number compared to the max??

np.float32(-0.3300379)

In [75]:
np.min(I_xy_shape_normalization)

np.float32(-201936.39)

In [56]:
np.max(I_xx_shape_normalization) #this number keeps getting bigger with the normalizations but same order of mag

np.float32(293710.53)

In [34]:
S_ij_trace = I_xx_shape_normalization + I_yy_shape_normalization + I_zz_shape_normalization

I_xx_trace_free = I_xx_shape_normalization - (1/3) * S_ij_trace

I_xy_trace_free = I_xy_shape_normalization

I_xz_trace_free = I_xz_shape_normalization

I_yy_trace_free = I_yy_shape_normalization - (1/3) * S_ij_trace

I_yz_trace_free = I_yz_shape_normalization

I_zz_trace_free = I_zz_shape_normalization - (1/3) * S_ij_trace

In [51]:
np.min(I_xx_trace_free)

np.float32(-188795.34)

In [38]:
I_xx_delta_fft = np.fft.rfftn(I_xx_trace_free)
I_xx_power_3d = np.conj(I_xx_delta_fft)*I_xx_delta_fft * L_box ** 3
I_xx_power_3d_flattened = I_xx_power_3d.flatten()

I_xy_delta_fft = np.fft.rfftn(I_xy_trace_free)
I_xy_power_3d = np.conj(I_xy_delta_fft)*I_xy_delta_fft * L_box ** 3
I_xy_power_3d_flattened = I_xy_power_3d.flatten()

I_xz_delta_fft = np.fft.rfftn(I_xz_trace_free)
I_xz_power_3d = np.conj(I_xz_delta_fft)*I_xz_delta_fft * L_box ** 3
I_xz_power_3d_flattened = I_xz_power_3d.flatten()

I_yy_delta_fft = np.fft.rfftn(I_yy_trace_free)
I_yy_power_3d = np.conj(I_yy_delta_fft)*I_yy_delta_fft * L_box ** 3
I_yy_power_3d_flattened = I_yy_power_3d.flatten()

I_yz_delta_fft = np.fft.rfftn(I_yz_trace_free)
I_yz_power_3d = np.conj(I_yz_delta_fft)*I_yz_delta_fft * L_box ** 3
I_yz_power_3d_flattened = I_yz_power_3d.flatten()

I_zz_delta_fft = np.fft.rfftn(I_zz_trace_free)
I_zz_power_3d = np.conj(I_zz_delta_fft)*I_zz_delta_fft * L_box ** 3
I_zz_power_3d_flattened = I_zz_power_3d.flatten()

In [78]:
np.max(I_xx_power_3d)

np.complex64(4.374895e+20-921799800000j)

In [41]:
np.random.seed(42)
three_numbers = np.random.rand(3)
magnitude = np.linalg.norm(three_numbers)
n_hat = three_numbers / magnitude

In [42]:
n_hat

array([0.29797254, 0.75635891, 0.58235175])

In [43]:
k = np.where(k == 0, 1e-10, k)
kx_hat = kx / k
ky_hat = ky / k
kz_hat = kz / k

In [44]:
#k = np.stack((kx, ky, kz), axis=-1)
k_hat = np.stack((kx_hat, ky_hat, kz_hat), axis=-1)
#k_cross_n = np.cross(k_hat, n_hat, axis=-1)
#k_cross_n_mag = np.linalg.norm(k_cross_n, axis=-1, keepdims=True)
#k_cross_n_mag[k_cross_n_mag == 0] = 1
#e_1 = k_cross_n / k_cross_n_mag

In [28]:
e_1.shape 
#???????????

(512, 512, 257, 3)

In [29]:
e_k = k_hat
#e_2_lite = np.cross(k_hat[0], e_1[0], axis=-1)

In [30]:
k_hat = np.stack((kx_hat, ky_hat, kz_hat), axis=-1)
k_hat.shape

(512, 512, 257, 3)

In [36]:
e_2 = np.cross(k_hat, e_1, axis=-1)

In [38]:
np.savez('helicity_basis.npz', array1=e_1, array2=e_2, array3=e_k)

In [45]:
basis_vectors = np.load('helicity_basis.npz')
e_1 = basis_vectors['array1']
e_2 = basis_vectors['array2']
e_k = basis_vectors['array3']

In [ ]:
inertia_flattened = 

In [24]:
np.savez('k_hat.npz', array1=k_hat, array2=e_plus, array3=e_minus)

In [25]:
np.savez('inertia_flattened.npz', array1=I_xx_power_3d_flattened, array2=I_xy_power_3d_flattened, array3=I_xz_power_3d_flattened, array4=I_yy_power_3d_flattened, array5=I_yz_power_3d_flattened, array6=I_zz_power_3d_flattened)

In [22]:
e_plus = (1 / np.sqrt(2)) * (e_1 - 1j * e_2)
e_minus = (1 / np.sqrt(2)) * (e_1 + 1j * e_2)

In [26]:
I_00 = np.sqrt(1/3) * (I_xx_power_3d_flattened + I_yy_power_3d_flattened + I_zz_power_3d_flattened)

In [27]:
I_20 = np.sqrt(3/2) * (k_hat[:,:, :, 0].flatten() ** 2 - (1/3)) * I_xx_power_3d_flattened
I_20 += np.sqrt(3/2) * (k_hat[:,:, :, 1].flatten() ** 2 - (1/3))  * I_yy_power_3d_flattened
I_20 += np.sqrt(3/2) * (k_hat[:,:, :, 2].flatten() ** 2 - (1/3)) * I_zz_power_3d_flattened
I_20 += 2 * np.sqrt(3/2) * (k_hat[:,:, :, 0].flatten() * k_hat[:,:, :, 1].flatten() * I_xy_power_3d_flattened)
I_20 += 2 * np.sqrt(3/2) * (k_hat[:,:, :, 0].flatten() * k_hat[:,:, :, 2].flatten() * I_xz_power_3d_flattened)
I_20 += 2 * np.sqrt(3/2) * (k_hat[:,:, :, 1].flatten() * k_hat[:,:, :, 2].flatten() * I_yz_power_3d_flattened)

In [28]:
I_21minus = np.sqrt(1/2) * (k_hat[:,:, :, 0].flatten() * e_plus[:,:, :, 0].flatten() + k_hat[:,:, :, 0].flatten() * e_plus[:,:, :, 0].flatten()) *  I_xx_power_3d_flattened
I_21minus += 2 * np.sqrt(1/2) * (k_hat[:,:, :, 0].flatten() * e_plus[:,:, :, 1].flatten() + k_hat[:,:, :, 1].flatten() * e_plus[:,:, :, 0].flatten()) * I_xy_power_3d_flattened
I_21minus += 2 * np.sqrt(1/2) * (k_hat[:,:, :, 0].flatten() * e_plus[:,:, :, 2].flatten() + k_hat[:,:, :, 2].flatten() * e_plus[:,:, :, 0].flatten()) * I_xz_power_3d_flattened
I_21minus += np.sqrt(1/2) * (k_hat[:,:, :, 1].flatten() * e_plus[:,:, :, 1].flatten() + k_hat[:,:, :, 1].flatten() * e_plus[:,:, :, 1].flatten()) * I_yy_power_3d_flattened
I_21minus += 2 * np.sqrt(1/2) * (k_hat[:,:, :, 1].flatten() * e_plus[:,:, :, 2].flatten() + k_hat[:,:, :, 2].flatten() * e_plus[:,:, :, 1].flatten()) * I_yz_power_3d_flattened
I_21minus += np.sqrt(1/2) * (k_hat[:,:, :, 2].flatten() * e_plus[:,:, :, 2].flatten() + k_hat[:,:, :, 2].flatten() * e_plus[:,:, :, 2].flatten()) * I_zz_power_3d_flattened                       

In [29]:
I_21plus = np.sqrt(1/2) * (k_hat[:,:, :, 0].flatten() * e_minus[:,:, :, 0].flatten() + k_hat[:,:, :, 0].flatten() * e_minus[:,:, :, 0].flatten()) * I_xx_power_3d_flattened
I_21plus += np.sqrt(1/2) * (k_hat[:,:, :, 0].flatten() * e_minus[:,:, :, 1].flatten() + k_hat[:,:, :, 1].flatten() * e_minus[:,:, :, 0].flatten()) * I_xy_power_3d_flattened
I_21plus += np.sqrt(1/2) * (k_hat[:,:, :, 0].flatten() * e_minus[:,:, :, 2].flatten() + k_hat[:,:, :, 2].flatten() * e_minus[:,:, :, 0].flatten()) * I_xz_power_3d_flattened
I_21plus += np.sqrt(1/2) * (k_hat[:,:, :, 1].flatten() * e_minus[:,:, :, 1].flatten() + k_hat[:,:, :, 1].flatten() * e_minus[:,:, :, 1].flatten()) * I_yy_power_3d_flattened
I_21plus += np.sqrt(1/2) * (k_hat[:,:, :, 1].flatten() * e_minus[:,:, :, 2].flatten() + k_hat[:,:, :, 2].flatten() * e_minus[:,:, :, 1].flatten()) * I_yz_power_3d_flattened
I_21plus += np.sqrt(1/2) * (k_hat[:,:, :, 2].flatten() * e_minus[:,:, :, 2].flatten() + k_hat[:,:, :, 2].flatten() * e_minus[:,:, :, 2].flatten()) * I_zz_power_3d_flattened                       

In [30]:
I_22minus = (e_plus[:,:, :, 0].flatten() * e_plus[:,:, :, 0].flatten()) * I_xx_power_3d_flattened
I_22minus += (e_plus[:,:, :, 0].flatten() * e_plus[:,:, :, 1].flatten()) * I_xy_power_3d_flattened
I_22minus += (e_plus[:,:, :, 0].flatten() * e_plus[:,:, :, 2].flatten()) * I_xz_power_3d_flattened
I_22minus += (e_plus[:,:, :, 1].flatten() * e_plus[:,:, :, 1].flatten()) * I_yy_power_3d_flattened
I_22minus += (e_plus[:,:, :, 1].flatten() * e_plus[:,:, :, 2].flatten()) * I_yz_power_3d_flattened
I_22minus += (e_plus[:,:, :, 2].flatten() * e_plus[:,:, :, 2].flatten()) * I_zz_power_3d_flattened

In [31]:
I_22plus = (e_minus[:,:, :, 0].flatten() * e_minus[:,:, :, 0].flatten()) * I_xx_power_3d_flattened
I_22plus += (e_minus[:,:, :, 0].flatten() * e_minus[:,:, :, 1].flatten()) * I_xy_power_3d_flattened
I_22plus += (e_minus[:,:, :, 0].flatten() * e_minus[:,:, :, 2].flatten()) * I_xz_power_3d_flattened
I_22plus += (e_minus[:,:, :, 1].flatten() * e_minus[:,:, :, 1].flatten()) * I_yy_power_3d_flattened
I_22plus += (e_minus[:,:, :, 1].flatten() * e_minus[:,:, :, 2].flatten()) * I_yz_power_3d_flattened
I_22plus += (e_minus[:,:, :, 2].flatten() * e_minus[:,:, :, 2].flatten()) * I_zz_power_3d_flattened

In [74]:
k.shape

(512, 512, 257, 3)

In [77]:
import gc

a = np.arange(10)
del a
gc.collect()

803

In [80]:
I_00.shape

(67371008,)

In [ ]:
#remember factor of two in the non diagonal terms, and then switch my plus minus components b/c i'm taking conjugate
#P(00) is delta galaxy, use d_gal not P_00

In [32]:
np.savez('inertia_legendres', array1=I_00, array2=I_20, array3=I_21minus, array4=I_21plus, array5=I_22minus, array6=I_22plus)

In [33]:
from abacusnbody.analysis.power_spectrum import calc_pk_from_deltak

In [34]:
from scipy import stats
bin_means, bin_edges, binnumber = stats.binned_statistic(
    k_flat, I_00, statistic='mean', bins=k_space_bins)

In [55]:
power_0020 = calc_pk_from_deltak(field_fft=I_00.reshape(512, 512, 512//2+1), Lbox=500, k_bin_edges=bin_edges, mu_bin_edges=np.array([0, 1]))#, field2_fft=I_00.reshape(512, 512, 512//2+1))

In [39]:
N_gal.shape

(512, 512, 512)

In [40]:
I_20.shape

(67371008,)

In [42]:
I_00.shape

(67371008,)

In [56]:
power_0020

{'power': array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf], dtype=float32),
 'N_mode': array([    27,    224,    644,   1208,   1964,   3056,   4000,   5756,
          6992,   9116,  10832,  13232,  15404,  17992,  20876,  23720,
         26888,  30236,  34208,  37412,  41560,  45992,  50132,  54392,
         59012,  65216,  68848,  75356,  79448,  87044,  91664,  98300,
        105152, 110488, 117140, 125696, 132284, 138272, 146720, 154364]),
 'binned_poles': array([], shape=(0, 40), dtype=float32),
 'N_mode_poles': array([    27,    224,    644,   1208,   1964,   3056,   4000,   5756,
          6992,   9116,  10832,  13232,  15404,  17992,  20876,  23720,
         26888,  30236,  34208,  37412,  41560,  45992,  50132,  54392,
         59012,  65216,  68848,  75356,  79448,  87044,  91664,  98300,
        

In [49]:
calc_pk_from_deltak

<function abacusnbody.analysis.power_spectrum.calc_pk_from_deltak(field_fft, Lbox, k_bin_edges, mu_bin_edges, field2_fft=None, poles=array([], dtype=int64), squeeze_mu_axis=True, nthread=10)>

In [50]:
bin_edges

array([0.   , 0.025, 0.05 , 0.075, 0.1  , 0.125, 0.15 , 0.175, 0.2  ,
       0.225, 0.25 , 0.275, 0.3  , 0.325, 0.35 , 0.375, 0.4  , 0.425,
       0.45 , 0.475, 0.5  , 0.525, 0.55 , 0.575, 0.6  , 0.625, 0.65 ,
       0.675, 0.7  , 0.725, 0.75 , 0.775, 0.8  , 0.825, 0.85 , 0.875,
       0.9  , 0.925, 0.95 , 0.975, 1.   ])

In [51]:
I_00.reshape(512, 512, 512//2+1)

array([[[1.35337572e+12+0.00000000e+00j, 1.26701484e+20-5.98887179e+11j,
         1.01390423e+20+1.34719417e+12j, ...,
         6.83646563e+18-2.17936413e+09j, 4.45545798e+18-4.61892039e+09j,
         3.93259193e+19+0.00000000e+00j],
        [2.75134988e+19-6.48138933e+10j, 1.04537605e+20-1.53872009e+11j,
         2.03585022e+19-3.59916763e+10j, ...,
         2.36676063e+19+6.25627485e+10j, 3.74857019e+18-1.05192694e+09j,
         2.80190613e+19+1.64351180e+11j],
        [7.20003489e+19+7.35886244e+11j, 1.38024253e+20+1.32503821e+12j,
         5.24027715e+19-7.30266507e+11j, ...,
         1.83885056e+19-1.59194245e+10j, 1.19589864e+19-2.30667041e+10j,
         1.05073740e+19-2.76002826e+10j],
        ...,
        [5.38608640e+19+2.78829338e+11j, 9.80661356e+19-3.74984176e+11j,
         7.89836680e+19+2.48972569e+11j, ...,
         2.32703997e+19-6.68665203e+10j, 3.08428405e+18-3.14863527e+10j,
         1.22849630e+19-1.25007755e+10j],
        [7.20003489e+19-7.35886244e+11j, 3.10047394